In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
import pickle

In [2]:
df_train = pd.read_csv('./data/train_final.csv')
df_train.drop(['education'], axis=1, inplace=True)
workclass_maxf = df_train['workclass'].value_counts().index[0]
occupation_maxf = df_train['occupation'].value_counts().index[0]
native_country_maxf = df_train['native.country'].value_counts().index[0]
df_train = df_train.replace('?', np.nan)
df_train['workclass'].fillna(workclass_maxf, inplace=True)
df_train['occupation'].fillna(occupation_maxf, inplace=True)
df_train['native.country'].fillna(native_country_maxf, inplace=True)
df_train

,age,workclass,fnlwgt,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income>50K
0,53,Self-emp-not-inc,93449,15,Married-civ-spouse,Prof-specialty,Husband,Asian-Pac-Islander,Male,0,0,40,India,1
1,33,Self-emp-not-inc,123424,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,40,United-States,1
2,47,Private,144844,9,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,United-States,0
3,40,Private,114580,9,Divorced,Craft-repair,Other-relative,White,Female,0,0,40,Vietnam,0
4,39,Private,115618,9,Married-civ-spouse,Transport-moving,Husband,White,Male,0,0,50,United-States,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24995,18,Private,83451,9,Never-married,Adm-clerical,Not-in-family,White,Female,0,0,25,United-States,0
24996,64,Local-gov,202738,9,Married-civ-spouse,Adm-clerical,Wife,White,Female,0,0,35,United-States,0
24997,39,Private,225544,14,Married-civ-spouse,Transport-moving,Husband,White,Male,0,0,40,Poland,0
24998,53,Private,346871,9,Divorced,Prof-specialty,Not-in-family,White,Male,4787,0,46,United-States,1


In [3]:
df_test = pd.read_csv('./data/test_final.csv')
df_test.drop(['ID', 'education'], axis=1, inplace=True)
df_test = df_test.replace('?', np.nan)
df_test['workclass'].fillna(workclass_maxf, inplace=True)
df_test['occupation'].fillna(occupation_maxf, inplace=True)
df_test['native.country'].fillna(native_country_maxf, inplace=True)
df_test

,age,workclass,fnlwgt,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country
0,33,Self-emp-not-inc,222162,9,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,United-States
1,68,Private,29240,9,Widowed,Prof-specialty,Not-in-family,White,Female,0,0,12,United-States
2,34,Private,103596,9,Married-civ-spouse,Handlers-cleaners,Husband,White,Male,0,0,40,United-States
3,57,Private,103403,3,Married-civ-spouse,Transport-moving,Husband,White,Male,0,0,40,United-States
4,48,Private,152915,10,Never-married,Adm-clerical,Not-in-family,White,Female,0,0,40,United-States
...,...,...,...,...,...,...,...,...,...,...,...,...,...
23837,26,Private,43408,13,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,40,United-States
23838,27,Private,116372,13,Never-married,Prof-specialty,Not-in-family,White,Female,0,0,40,United-States
23839,67,Self-emp-inc,182581,10,Married-civ-spouse,Exec-managerial,Husband,White,Male,20051,0,20,United-States
23840,46,Local-gov,274689,12,Divorced,Protective-serv,Not-in-family,White,Male,0,0,40,United-States


In [4]:
x_train, y_train = df_train.iloc[:, :-1], df_train.iloc[:, -1]
x_combn = x_train.append(df_test)
x_combn = pd.get_dummies(x_combn)
for col in ['age', 'fnlwgt', 'education.num', 'capital.gain', 'capital.loss', 'hours.per.week']:
    x_combn[col] = x_combn[col]/x_combn[col].max()
x_train, x_test = x_combn.iloc[:len(y_train),:], x_combn.iloc[len(y_train):,:]

/tmp/ipykernel_719206/2692513763.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  x_combn = x_train.append(df_test)


In [25]:
model = RandomForestClassifier(n_estimators = 256, max_depth=4)
model = model.fit(x_train, y_train)

In [26]:
print(100*sum(model.predict(x_train) == y_train)/len(y_train))

82.252


In [27]:
ans = pd.DataFrame.from_dict({
    'ID': np.arange(1, len(x_test) + 1),
    'Prediction': model.predict(x_test),
})
ans.to_csv('rf.csv', index=False)